In [1]:
import pandas as pd

In [2]:
# Defining the file paths
output_folder_path = '/cluster/scratch/salathem/data/output_euler/outputDRT_test_cost/'
drt_vehicle_stats_path = output_folder_path + 'drt_vehicle_stats_drt.csv'
drt_passenger_stats_path = output_folder_path + 'drt_customer_stats_drt.csv'
output_file_path = 'drt_operator_costs.csv'

In [10]:
# Cost parameters
car_cost_per_km = 0.7  # CHF/km for car
pt_cost_per_km = 0.6   # CHF/km for public transport
pt_min_cost = 2.7      # Minimum cost for public transport
pt_regional_radius_km = 15.0  # Regional radius in km for public transport

# DRT specific cost parameters
drt_cost_per_distance = 0.098   # CHF per km
drt_cost_per_trip = 0.375       # CHF per trip
drt_cost_per_vehicle = 33.30    # CHF per vehicle

# DRT specific revenue parameters
drt_revenue_per_trip = 2.0      # CHF per trip
drt_revenue_per_distance = 0.5 # CHF per km

In [4]:
drt_vehicle_data = pd.read_csv(drt_vehicle_stats_path, delimiter=';')
drt_person_data = pd.read_csv(drt_passenger_stats_path, delimiter=';')


In [5]:
# Merging the two dataframes on 'iteration' column
merged_data = pd.merge(drt_person_data, drt_vehicle_data, on="iteration")

In [11]:
# Calculating the revenues
merged_data["revenue_trip"] = merged_data["rides"] * drt_revenue_per_trip
merged_data["revenue_distance"] = (merged_data["totalPassengerDistanceTraveled"] / 1000) * drt_revenue_per_distance
merged_data["revenue_total"] = merged_data["revenue_trip"] + merged_data["revenue_distance"]
# Calculating the costs
merged_data["cost_distance"] = (merged_data["totalDistance"] / 1000) * drt_cost_per_distance
merged_data["cost_trip"] = merged_data["rides"] * drt_cost_per_trip
merged_data["cost_vehicle"] = merged_data["vehicles"] * drt_cost_per_vehicle
merged_data["cost_total"] = merged_data["cost_distance"] + merged_data["cost_trip"] + merged_data["cost_vehicle"]

# Creating the operator_cost dataframe with the calculated revenues
operator_cost = merged_data[["iteration", "revenue_trip", "revenue_distance", "revenue_total", "cost_distance", "cost_trip", "cost_vehicle", "cost_total"]]

 # Displaying the first few rows of the dataframe

In [12]:
operator_cost["win_loss"] = operator_cost["revenue_total"] - operator_cost["cost_total"]


/tmp/ipykernel_24821/3609016174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operator_cost["win_loss"] = operator_cost["revenue_total"] - operator_cost["cost_total"]


In [13]:
operator_cost

,iteration,revenue_trip,revenue_distance,revenue_total,cost_distance,cost_trip,cost_vehicle,cost_total,win_loss
0,0,0.0,0.000000,0.000000,0.000000,0.000,3330.0,3330.000000,-3330.000000
1,1,244.0,307.568605,551.568605,78.422842,45.750,3330.0,3454.172842,-2902.604237
2,2,518.0,646.763995,1164.763995,163.089705,97.125,3330.0,3590.214705,-2425.450710
3,3,758.0,994.853265,1752.853265,251.562975,142.125,3330.0,3723.687975,-1970.834710
4,4,978.0,1269.229090,2247.229090,320.363658,183.375,3330.0,3833.738658,-1586.509568
5,5,1180.0,1522.995125,2702.995125,390.500139,221.250,3330.0,3941.750139,-1238.755014
6,6,1320.0,1733.196880,3053.196880,442.141924,247.500,3330.0,4019.641924,-966.445044
7,7,1474.0,1985.499860,3459.499860,510.760959,276.375,3330.0,4117.135959,-657.636099
8,8,1632.0,2184.700260,3816.700260,566.881605,306.000,3330.0,4202.881605,-386.181345
9,9,1730.0,2316.853385,4046.853385,599.868962,324.375,3330.0,4254.243962,-207.390577


In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filter data for the last iteration only
last_iteration = operator_cost[operator_cost["iteration"] == operator_cost["iteration"].max()]

# Create subplots
fig = make_subplots(rows=3, cols=1, vertical_spacing=0.15)

# Stacked bar chart for costs
fig.add_trace(
    go.Bar(x=last_iteration["iteration"], y=last_iteration["cost_distance"], name="Cost Distance"),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=last_iteration["iteration"], y=last_iteration["cost_trip"], name="Cost Trip"),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=last_iteration["iteration"], y=last_iteration["cost_vehicle"], name="Cost Vehicle"),
    row=1, col=1
)

# Stacked bar chart for revenue
fig.add_trace(
    go.Bar(x=last_iteration["iteration"], y=last_iteration["revenue_trip"], name="Revenue Trip"),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=last_iteration["iteration"], y=last_iteration["revenue_distance"], name="Revenue Distance"),
    row=2, col=1
)

# Bar chart for win/loss
fig.add_trace(
    go.Bar(x=last_iteration["iteration"], y=last_iteration["win_loss"], name="Win/Loss"),
    row=3, col=1
)

# Update y-axes labels
fig.update_yaxes(title_text="Costs (CHF)", row=1, col=1)
fig.update_yaxes(title_text="Revenue (CHF)", row=2, col=1)
fig.update_yaxes(title_text="Win/Loss (CHF)", row=3, col=1)

# Update layout
fig.update_layout(height=900, width=700, title_text="DRT Operator Costs and Revenue Analysis", barmode='stack')

# Show the plot
fig.show()



In [ ]:







# Calculating the operator cost for DRT
drt_data['OperatorCost'] = (drt_data['totalDistance'] * drt_cost_per_distance / 1000) + \
                            (drt_data['vehicles'] * drt_cost_per_vehicle)


# Saving the updated data to a new CSV file
drt_data.to_csv(output_file_path, index=False)


output_file_path
